<a href="https://colab.research.google.com/github/jdetras/dnaplace_DE_webscraping/blob/main/dnaplace_webscraping_DE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4

In [ ]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
#from google.colab import files

# Upload your file
#uploaded = files.upload()  # Upload cis_elements.txt
#filename = list(uploaded.keys())[0]

# Read URLs from the uploaded file
with open(filename, "r") as f:
    urls = [line.strip() for line in f if line.strip()]

# Function to extract all DE lines from the preformatted text
def extract_de(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")

        pre_tag = soup.find("pre")
        if not pre_tag:
            return None

        text_lines = pre_tag.get_text().splitlines()
        de_lines = []
        for line in text_lines:
            if line.startswith("DE"):
                # Remove the 'DE' marker and leading whitespace
                cleaned_line = line[2:].strip()
                de_lines.append(cleaned_line)

        if de_lines:
            return " ".join(de_lines)
        else:
            return None
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

# Extract DEs and store in a list
data = []
for url in urls:
    de = extract_de(url)
    data.append({"URL": url, "DE": de})

# Write to CSV
csv_filename = "DE_extracted.csv"
with open(csv_filename, "w", newline='', encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["URL", "DE"])
    writer.writeheader()
    writer.writerows(data)

print(f"CSV file '{csv_filename}' created successfully!")

# Auto-download CSV
files.download(csv_filename)